<a href="https://colab.research.google.com/github/dj-1087/BigContest-COIN/blob/master/LSTM_EX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AExKfseySvluBXI_hl7quFmg4FAJFlkLaS_6XvGifCuzeCTI4oh23o
Mounted at /content/gdrive


In [2]:
from datetime import datetime
def parse(x):
  return datetime.strptime(x, '%Y-%m-%d')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

PATH = "/content/gdrive/My Drive/Colab Notebooks/AfterMay_DF/"
team_hitter = pd.read_csv(PATH+"team_hitter.csv")
pitcher_r_vs = pd.read_csv(PATH+"pitcher_r_vs.csv")
pitcher_r = pd.read_csv(PATH+"pitcher_r.csv")
hitter_r_vs = pd.read_csv(PATH+"hitter_r_vs.csv")
hitter_r = pd.read_csv(PATH+"hitter_r.csv")

In [4]:
hitter_r_vs["일자"] = hitter_r_vs["일자"].apply(parse)
hitter_r_vs = hitter_r_vs.set_index("일자",)
hitter_r["일자"] = hitter_r["일자"].apply(parse)
hitter_r = hitter_r.set_index("일자")
pitcher_r_vs["일자"] = pitcher_r_vs["일자"].apply(parse)
pitcher_r_vs = pitcher_r_vs.set_index("일자")
pitcher_r["일자"] = pitcher_r["일자"].apply(parse)
pitcher_r = pitcher_r.set_index("일자")
hitter_r_vs.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4879 entries, 2016-05-11 to 2020-07-19
Data columns (total 67 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      4879 non-null   int64  
 1   게임키        4879 non-null   object 
 2   팀코드        4879 non-null   object 
 3   상대팀코드      4879 non-null   object 
 4   더블헤더코드     4879 non-null   int64  
 5   초말         4879 non-null   object 
 6   타자         4879 non-null   int64  
 7   타수         4879 non-null   int64  
 8   타점         4879 non-null   int64  
 9   득점         4879 non-null   int64  
 10  안타         4879 non-null   int64  
 11  2루타        4879 non-null   int64  
 12  3루타        4879 non-null   int64  
 13  홈런         4879 non-null   int64  
 14  도루         4879 non-null   int64  
 15  도루실패       4879 non-null   int64  
 16  희타         4879 non-null   int64  
 17  희비         4879 non-null   int64  
 18  4구         4879 non-null   int64  
 19  고4         4879 non-null   int

In [5]:
hitter_r_vs = hitter_r_vs.drop(labels=['index'],axis=1)
hitter_r = hitter_r.drop(labels=['index'],axis=1)
pitcher_r_vs = pitcher_r_vs.drop(labels=['index'],axis=1)
pitcher_r = pitcher_r.drop(labels=['index'],axis=1)

In [6]:
pitcher_r

,게임키,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,최근평균자책점,최근RA/9,최근ERC,최근FIP,최근kwERA,최근LOB,최근HR%,최근K%,최근BB%,최근K9,최근BB9,최근HR9,최근KBB,최근whip,최근PFR,최근P/G,최근P/IP,최근P/PA,최근CYP,최근F-E,최근R-E,최근OAVG,최근OOBP,최근OSLG,최근OOPS,단타,최근투수AB,최근투수HIT,최근투수B1B,최근투수B2B,최근투수B3B,최근투수BB,최근투수HR,최근투수K,최근투수HBP,최근투수IBB,최근투수SB,최근투수CS,최근투수PA,최근투수INN,최근투수실점,최근득점권WHIP,최근2점차이하WHIP,최근345번타자WHIP
일자,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-05-11,20160511KTHT0,HT,KT,0,B,0,W,0,27,146,37,34,9,1,0,1,0,0,0,0,2,0,1,9,3,0,0,0,3,3,1.200,1.000,1.000,2016,5.062,6.117,5.808,5.531,4.538,0.676,0.031,0.154,0.082,6.328,3.164,1.266,2.000,1.594,1.078,148.2,17.367,3.800,2.333,0.469,1.055,0.297,0.369,0.463,0.832,7,35.0,10.4,7.6,1.0,0.6,3.0,1.2,6.0,0.8,0.2,1.4,0.6,39.0,8.533333,5.8,2.1428,4.6862,2.0002
2016-05-11,20160511KTHT0,KT,HT,0,T,0,L,0,24,156,40,38,14,6,0,1,3,0,1,0,1,0,0,6,0,0,0,1,8,6,2.100,3.429,3.000,2016,6.400,8.000,6.254,5.840,4.951,0.571,0.028,0.154,0.117,6.600,5.000,1.200,1.320,1.778,1.289,174.4,19.378,4.075,-3.200,-0.560,1.600,0.301,0.387,0.492,0.879,7,36.6,11.0,6.8,2.6,0.4,5.0,1.2,6.6,0.4,0.0,0.8,0.0,42.8,9.000000,8.0,2.7238,1.2890,2.9000
2016-05-11,20160511NCHH0,HH,NC,0,B,0,W,2,27,187,45,35,9,1,0,0,0,0,0,1,9,0,0,8,1,0,1,0,5,4,2.100,2.000,4.000,2016,11.700,13.050,13.516,8.048,5.668,0.542,0.045,0.116,0.138,5.850,6.975,2.250,0.839,2.750,1.425,167.0,20.875,3.728,-29.400,-3.652,1.350,0.418,0.498,0.619,1.117,8,33.4,11.4,9.4,0.8,0.2,5.8,1.0,5.6,0.2,0.0,0.8,0.6,40.4,8.400000,6.4,2.2822,1.9540,2.3050
2016-05-11,20160511NCHH0,NC,HH,0,T,0,L,0,24,150,40,36,13,4,0,0,0,0,1,0,3,0,0,7,1,0,0,1,6,5,1.875,1.941,3.000,2016,3.200,3.200,2.666,3.862,4.239,0.737,0.005,0.194,0.097,7.200,3.600,0.200,2.000,1.200,1.200,150.6,16.733,4.048,15.100,0.662,0.000,0.221,0.310,0.252,0.561,9,32.6,7.2,6.6,0.4,0.0,3.6,0.2,7.2,0.6,0.0,0.4,0.4,37.2,9.000000,3.2,1.5900,0.9470,1.1818
2016-05-11,20160511OBSK0,SK,OB,0,B,0,L,0,27,163,41,35,12,2,0,3,1,1,0,1,5,0,0,6,2,0,0,0,7,7,1.714,1.714,2.333,2016,5.757,6.551,5.959,6.486,4.891,0.689,0.048,0.137,0.094,5.757,3.176,1.985,1.812,1.632,1.081,151.8,16.743,3.580,-1.433,0.729,0.794,0.290,0.361,0.522,0.882,7,37.2,10.8,6.4,2.2,0.2,3.2,2.0,5.8,0.2,0.8,0.8,0.0,42.4,9.066667,6.6,1.5254,1.7020,1.7850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-19,20200719LTSS0,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600,1.111,1.500,2020,8.163,8.163,6.980,5.550,4.932,0.579,0.029,0.132,0.093,5.651,3.767,1.256,1.500,1.837,1.070,162.4,18.884,3.961,-12.800,-2.613,0.000,0.337,0.401,0.500,0.901,6,35.6,12.0,8.8,1.8,0.2,3.6,1.2,5.4,0.4,0.2,1.2,0.2,41.0,8.600000,7.8,2.5466,2.0584,1.6834
2020-07-19,20200719OBHT0,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.143,1.800,0.750,2020,4.740,4.740,5.472,5.589,5.147,0.754,0.026,0.132,0.111,5.153,4.328,1.031,1.190,1.580,1.053,150.4,17.221,3.958,3.333,0.849,0.000,0.304,0.387,0.424,0.811,7,31.6,9.6,7.8,0.8,0.0,4.2,1.0,5.0,0.6,0.0,0.6,0.6,38.0,8.733333,4.6,1.1722,1.6374,1.9614
2020-07-19,20200719OBHT0,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000,1.000,1.875,2020,3.477,4.091,4.394,4.142,4.348,0.748,0.016,0.170,0.082,6.750,3.068,0.614,2.200,1.477,1.114,150.2,17.068,3.871,11.300,0.665,0.614,0.285,0.349,0.360,0.709,7,34.4,9.8,8.4,0.8,0.0,3.0,0.6,6.6,0.4,0.2,1.4,0.2,38.8,8.800000,4.0,1.1250,1.7200,1.5836


In [ ]:
pitcher_r_vs

,게임키,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,상대최근평균자책점,상대최근RA/9,상대최근ERC,상대최근FIP,상대최근LOB,상대최근홈런율,상대최근삼진율,상대최근K9,상대최근BB9,상대최근HR9,상대최근KBB,상대최근whip,상대최근PFR,상대최근P/G,상대최근P/IP,상대최근P/PA,상대최근CYP,상대최근kwERA,상대최근K%,상대최근BB%,상대최근OAVG,상대최근OOBP,단타,상대최근OSLG,상대최근OOPS,상대최근투수AB,상대최근투수HIT,상대최근투수B1B,상대최근투수B2B,상대최근투수B3B,상대최근투수BB,상대최근투수HR,상대최근투수K,상대최근투수HBP,상대최근투수IBB,상대최근투수SB,상대최근투수CS,상대최근투수INN,상대최근득점권WHIP,상대최근2점차이하WHIP,상대최근345번타자WHIP,상대최근투수실점
일자,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-05-11,20160511KTHT0,HT,KT,0,B,0,W,0,27,146,37,34,9,1,0,1,0,0,0,0,2,0,1,9,3,0,0,0,3,3,1.200,1.000,1.000,2016,3.960000,5.760000,4.842454,4.309841,0.645161,0.017391,0.173913,7.200000,2.880000,0.720000,2.500000,1.520000,1.120000,140.666667,16.880000,3.669565,4.500000,4.147826,0.173913,0.069565,0.291262,0.353982,7,0.417476,0.771458,34.333333,10.000000,7.000000,2.333333,0.000000,2.666667,0.666667,6.666667,0.666667,0.000000,1.333333,0.000000,8.333333,1.461667,1.811000,1.775000,5.333333
2016-05-11,20160511KTHT0,KT,HT,0,T,0,L,0,24,156,40,38,14,6,0,1,3,0,1,0,1,0,0,6,0,0,0,1,8,6,2.100,3.429,3.000,2016,3.333333,4.000000,3.660377,3.966138,0.718085,0.008621,0.189655,7.333333,4.000000,0.333333,1.833333,1.407407,1.259259,151.666667,16.851852,3.922414,7.700000,4.365517,0.189655,0.103448,0.254902,0.336207,7,0.372549,0.708756,34.000000,8.666667,6.000000,1.666667,0.666667,4.000000,0.333333,7.333333,0.333333,0.000000,1.333333,0.333333,9.000000,1.300000,1.157667,0.759667,4.000000
2016-05-11,20160511NCHH0,HH,NC,0,B,0,W,2,27,187,45,35,9,1,0,0,0,0,0,1,9,0,0,8,1,0,1,0,5,4,2.100,2.000,4.000,2016,5.400000,5.760000,4.566582,5.509841,0.660920,0.026549,0.168142,6.840000,4.680000,1.080000,1.461538,1.480000,1.280000,156.000000,18.720000,4.141593,0.400000,4.762832,0.168142,0.115044,0.252632,0.351351,8,0.389474,0.740825,31.666667,8.000000,5.666667,1.333333,0.000000,4.333333,1.000000,6.333333,0.666667,0.000000,1.666667,0.333333,8.333333,1.857000,1.435333,2.643000,5.333333
2016-05-11,20160511NCHH0,NC,HH,0,T,0,L,0,24,150,40,36,13,4,0,0,0,0,1,0,3,0,0,7,1,0,0,1,6,5,1.875,1.941,3.000,2016,2.333333,2.333333,2.434538,3.669841,0.822785,0.009174,0.266055,9.666667,4.000000,0.333333,2.416667,1.111111,1.518519,132.666667,14.740741,3.651376,11.400000,3.528440,0.266055,0.110092,0.202247,0.317308,9,0.269663,0.586971,29.666667,6.000000,4.666667,1.000000,0.000000,4.000000,0.333333,9.666667,1.000000,0.000000,0.000000,0.333333,9.000000,1.650000,0.966667,1.514333,2.333333
2016-05-11,20160511OBSK0,SK,OB,0,B,0,L,0,27,163,41,35,12,2,0,3,1,1,0,1,5,0,0,6,2,0,0,0,7,7,1.714,1.714,2.333,2016,5.468354,5.468354,6.023969,6.859715,0.766871,0.052174,0.104348,4.101266,1.708861,2.050633,2.400000,1.367089,0.645570,148.666667,16.936709,3.878261,-0.633333,4.669565,0.104348,0.043478,0.303922,0.362832,7,0.539216,0.902048,34.000000,10.333333,6.333333,2.000000,0.000000,1.666667,2.000000,4.000000,1.666667,0.000000,0.666667,0.000000,8.777778,0.908000,1.133667,1.847333,5.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-19,20200719LTSS0,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600,1.111,1.500,2020,8.280000,8.280000,6.667779,6.423343,0.567568,0.042017,0.109244,4.680000,3.600000,1.800000,1.300000,1.720000,0.920000,152.333333,18.280000,3.840336,-8.200000,5.097479,0.109244,0.084034,0.317308,0.376068,6,0.500000,0.876068,34.666667,11.000000,8.000000,1.333333,0.000000,3.333333,1.666667,4.333333,0.333333,0.000000,2.000000,0.333333,8.333333,2.194333,1.876333,1.255667,7.666667
2020-07-19,20200719OBHT0,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.143,1.800,0.750,2020,3.807692,3.807692,4.449642,4.235651,0.74418

In [ ]:
hitter_r_vs

,게임키,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,상대최근OBP,상대최근SLG,상대최근ops,상대최근타율,상대최근ISO,상대최근BABIP,상대최근BB/K,상대최근K%,상대최근BB%,상대최근wOBA,상대최근wRAA,상대최근wRC,상대최근HR%,상대최근GPA,상대최근RC,상대최근RC27,상대최근XR,상대최근SPD,상대최근SecA,상대최근PSN,상대최근PA,상대최근AB,상대최근HIT,상대최근B1B,상대최근B2B,상대최근B3B,상대최근BB,상대최근HR,상대최근K,상대최근HBP,상대최근IBB,상대최근GDP,상대최근SB,상대최근CS,상대최근득점권타수,상대최근득점권안타,상대최근득점권타율
일자,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-05-11,20160511KTHT0,HT,KT,0,B,40,38,6,8,14,6,0,1,3,0,1,0,1,0,0,6,0,1,8,0.375,16,6,2016,7,0.336207,0.372549,0.708756,0.254902,0.117647,0.312500,0.545455,0.189655,0.103448,0.328716,-3.519599,12.736589,0.008621,0.244430,12.372966,4.074025,12.814,5.614896,0.264706,1.600000,38.666667,34.000000,8.666667,6.000000,1.666667,0.666667,4.000000,0.333333,7.333333,0.333333,0.000000,1.333333,1.333333,0.333333,7.666667,2.333333,0.304348
2016-05-11,20160511KTHT0,KT,HT,0,T,37,34,3,3,9,1,0,1,0,0,0,0,2,0,1,9,3,1,7,0.333,6,2,2016,7,0.353982,0.417476,0.771458,0.291262,0.126214,0.345679,0.400000,0.173913,0.069565,0.354469,-0.789482,15.326566,0.017391,0.263661,13.933300,4.702489,14.040,3.261458,0.242718,2.666667,38.333333,34.333333,10.000000,7.000000,2.333333,0.000000,2.666667,0.666667,6.666667,0.666667,0.000000,1.666667,1.333333,0.000000,12.333333,2.666667,0.216216
2016-05-11,20160511NCHH0,HH,NC,0,B,40,36,5,6,13,4,0,0,0,0,1,0,3,0,0,7,1,3,10,0.333,12,4,2016,9,0.317308,0.269663,0.586971,0.202247,0.067416,0.288136,0.413793,0.266055,0.110092,0.290567,-7.097688,8.177523,0.009174,0.210204,6.964873,2.321624,7.478,0.572540,0.191011,0.000000,36.333333,29.666667,6.000000,4.666667,1.000000,0.000000,4.000000,0.333333,9.666667,1.000000,0.000000,1.333333,0.000000,0.333333,6.000000,1.333333,0.222222
2016-05-11,20160511NCHH0,NC,HH,0,T,45,35,4,5,9,1,0,0,0,0,0,1,9,0,0,8,1,0,13,0.273,11,3,2016,8,0.351351,0.389474,0.740825,0.252632,0.136842,0.283784,0.684211,0.168142,0.115044,0.343180,-1.938596,13.897173,0.026549,0.255477,13.213038,4.633143,14.548,4.734077,0.315789,3.750000,37.666667,31.666667,8.000000,5.666667,1.333333,0.000000,4.333333,1.000000,6.333333,0.666667,0.000000,0.666667,1.666667,0.333333,10.000000,2.666667,0.266667
2016-05-11,20160511OBSK0,SK,OB,0,B,35,32,3,3,9,2,0,0,0,0,1,0,2,0,0,7,1,1,5,0.250,8,2,2016,7,0.333333,0.405941,0.739274,0.267327,0.138614,0.328947,0.320000,0.215517,0.068966,0.336193,-2.728910,13.527278,0.017241,0.251485,12.688657,4.177972,13.950,3.240095,0.237624,2.400000,38.666667,33.666667,9.000000,5.666667,2.666667,0.000000,2.666667,0.666667,8.333333,1.000000,0.000000,1.000000,1.000000,0.333333,7.666667,1.666667,0.217391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-19,20200719LTSS0,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000,2,0,2020,4,0.353448,0.423077,0.776525,0.288462,0.134615,0.314607,0.714286,0.119658,0.085470,0.353526,0.865157,9.767532,0.017094,0.264821,15.096198,5.159460,15.628,4.605431,0.269231,2.666667,39.000000,34.666667,10.000000,6.666667,2.666667,0.000000,3.333333,0.666667,4.666667,0.333333,0.000000,1.000000,1.333333,0.000000,12.000000,3.666667,0.305556
2020-07-19,20200719OBHT0,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571,7,4,2020,7,0.370370,0.410526,0.780897,0.284211,0.126316,0.320988,0.923077,0.119266,0.100917,0.357150,1.159105,9.528860,0.009174,0.269298,13.445141,4.840251,12.956,2.587810,0.231579,0.000000,36.666667,31.666667,9.000000,6.000000,2.333333,0.333333,3.666667,0.333333,4.333333,0.333333,0.333333,1.000000,0.000000,0.666667,9.000000,0.666667,0.074074
2020-07-19,20200719OBHT0,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.364,11,4,2020,7,0.377193,0.336735,0.713928,0.295918,0.040816,0.329545,0.916667,0.103448,0.094828,0.341167,-0.385661,8.440626,0.000000,0.253921,12.869538,4.454840,13.394,2.82285

In [ ]:
cols_hitter = hitter_r_vs.columns.append(hitter_r.columns)
cols_pitcher = pitcher_r_vs.columns.append(pitcher_r.columns)

cols_hitter_duplicated = cols_hitter[cols_hitter.duplicated()==True].unique()
cols_pitcher_duplicated = cols_pitcher[cols_pitcher.duplicated()==True].unique()
cols = cols_hitter_duplicated.append(cols_pitcher_duplicated)
cols_duplicated = cols[cols.duplicated()==True].unique()
cols_pitcher_duplicated

Index(['게임키', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3', '투구수',
       '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구',
       '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '단타'],
      dtype='object')

In [ ]:
df_hitter = pd.merge(left=hitter_r_vs, right=hitter_r, how='left', on=['게임키','팀코드','상대팀코드','초말','시즌'],suffixes=('_상대최근','_최근'),left_index=True)
df_pitcher = pd.merge(left=pitcher_r_vs, right=pitcher_r, how='left', on=['게임키','팀코드','상대팀코드','초말','결과','시즌'],suffixes=('_상대최근','_최근'),left_index=True)
df = pd.merge(left=df_hitter, right=df_pitcher, how='left', on=['게임키','팀코드','상대팀코드','초말','시즌'],sort=True, suffixes=('_타자','_투수'),left_index=True)
df = df.drop(df[(df.결과=='D')].index)
df = df.dropna()
df

,게임키,팀코드,상대팀코드,더블헤더코드_상대최근_타자,초말,타자_상대최근_타자,타수_상대최근_타자,타점_상대최근,득점_상대최근,안타_상대최근_타자,2루타_상대최근_타자,3루타_상대최근_타자,홈런_상대최근_타자,도루_상대최근_타자,도루실패_상대최근_타자,희타_상대최근_타자,희비_상대최근_타자,4구_상대최근_타자,고4_상대최근_타자,사구_상대최근_타자,삼진_상대최근_타자,병살타_상대최근_타자,실책_상대최근_타자,잔루_상대최근,득점권타율_상대최근,득점권타수_상대최근,득점권안타_상대최근,시즌,단타_상대최근_타자,상대최근OBP,상대최근SLG,상대최근ops,상대최근타율,상대최근ISO,상대최근BABIP,상대최근BB/K,상대최근K%_타자,상대최근BB%_타자,상대최근wOBA,상대최근wRAA,...,최근kwERA,최근LOB,최근HR%_투수,최근K%_투수,최근BB%_투수,최근K9,최근BB9,최근HR9,최근KBB,최근whip,최근PFR,최근P/G,최근P/IP,최근P/PA,최근CYP,최근F-E,최근R-E,최근OAVG,최근OOBP,최근OSLG,최근OOPS,단타_최근_투수,최근투수AB,최근투수HIT,최근투수B1B,최근투수B2B,최근투수B3B,최근투수BB,최근투수HR,최근투수K,최근투수HBP,최근투수IBB,최근투수SB,최근투수CS,최근투수PA,최근투수INN,최근투수실점,최근득점권WHIP,최근2점차이하WHIP,최근345번타자WHIP
2016-05-11,20160511KTHT0,HT,KT,0,B,40,38,6,8,14,6,0,1,3,0,1,0,1,0,0,6,0,1,8,0.375,16,6,2016,7,0.336207,0.372549,0.708756,0.254902,0.117647,0.312500,0.545455,0.189655,0.103448,0.328716,-3.519599,...,4.538,0.676,0.031,0.154,0.082,6.328,3.164,1.266,2.000,1.594,1.078,148.2,17.367,3.800,2.333,0.469,1.055,0.297,0.369,0.463,0.832,7.0,35.0,10.4,7.6,1.0,0.6,3.0,1.2,6.0,0.8,0.2,1.4,0.6,39.0,8.533333,5.8,2.1428,4.6862,2.0002
2016-05-11,20160511KTHT0,KT,HT,0,T,37,34,3,3,9,1,0,1,0,0,0,0,2,0,1,9,3,1,7,0.333,6,2,2016,7,0.353982,0.417476,0.771458,0.291262,0.126214,0.345679,0.400000,0.173913,0.069565,0.354469,-0.789482,...,4.951,0.571,0.028,0.154,0.117,6.600,5.000,1.200,1.320,1.778,1.289,174.4,19.378,4.075,-3.200,-0.560,1.600,0.301,0.387,0.492,0.879,7.0,36.6,11.0,6.8,2.6,0.4,5.0,1.2,6.6,0.4,0.0,0.8,0.0,42.8,9.000000,8.0,2.7238,1.2890,2.9000
2016-05-11,20160511NCHH0,HH,NC,0,B,40,36,5,6,13,4,0,0,0,0,1,0,3,0,0,7,1,3,10,0.333,12,4,2016,9,0.317308,0.269663,0.586971,0.202247,0.067416,0.288136,0.413793,0.266055,0.110092,0.290567,-7.097688,...,5.668,0.542,0.045,0.116,0.138,5.850,6.975,2.250,0.839,2.750,1.425,167.0,20.875,3.728,-29.400,-3.652,1.350,0.418,0.498,0.619,1.117,8.0,33.4,11.4,9.4,0.8,0.2,5.8,1.0,5.6,0.2,0.0,0.8,0.6,40.4,8.400000,6.4,2.2822,1.9540,2.3050
2016-05-11,20160511NCHH0,NC,HH,0,T,45,35,4,5,9,1,0,0,0,0,0,1,9,0,0,8,1,0,13,0.273,11,3,2016,8,0.351351,0.389474,0.740825,0.252632,0.136842,0.283784,0.684211,0.168142,0.115044,0.343180,-1.938596,...,4.239,0.737,0.005,0.194,0.097,7.200,3.600,0.200,2.000,1.200,1.200,150.6,16.733,4.048,15.100,0.662,0.000,0.221,0.310,0.252,0.561,9.0,32.6,7.2,6.6,0.4,0.0,3.6,0.2,7.2,0.6,0.0,0.4,0.4,37.2,9.000000,3.2,1.5900,0.9470,1.1818
2016-05-11,20160511OBSK0,OB,SK,0,T,41,35,7,7,12,2,0,3,1,1,0,1,5,0,0,6,2,0,7,0.143,7,1,2016,7,0.362832,0.539216,0.902048,0.303922,0.235294,0.294118,0.416667,0.104348,0.043478,0.394398,3.396350,...,4.343,0.594,0.027,0.203,0.115,9.132,4.963,1.191,1.840,2.007,1.588,184.8,20.382,4.070,-14.733,-3.087,0.199,0.344,0.434,0.545,0.979,7.0,35.2,9.6,5.4,3.2,0.2,4.8,0.8,9.0,0.4,0.0,0.4,0.4,41.6,9.066667,5.6,1.7438,1.9974,1.8108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-19,20200719LTSS0,SS,LT,0,B,34,31,1,1,7,0,0,1,2,1,0,0,3,0,0,9,1,0,6,0.000,5,0,2020,6,0.376068,0.500000,0.876068,0.317308,0.182692,0.318182,0.769231,0.109244,0.084034,0.388906,4.531493,...,4.650,0.674,0.016,0.146,0.083,5.600,3.200,0.600,1.750,1.378,0.978,154.8,17.200,4.031,9.300,0.486,0.800,0.272,0.346,0.367,0.713,4.0,33.8,9.2,7.2,1.4,0.0,3.2,0.6,5.6,0.6,0.0,0.4,0.0,38.4,9.000000,4.8,1.7036,1.3554,1.5952
2020-07-19,20200719OBHT0,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571,7,4,2020,7,0.370370,0.410526,0.780897,0.284211,0.126316,0.320988,0.923077,0.119266,0.100917,0.357150,1.159105,...,5.147,0.754,0.026,0.132,0.111,5.153,4.328,1.031,1.190,1.580,1.053,150.4,17.221,3.958,3.333,0.849,0.000,0.304,0.387,0.424,0.811,7.0,31.6,9.6,7.8,0.8,0.0,4.2,1.0,5.0,0.6,0.0,0.6,0.6,38.0,8.733333,4.6,1.1722,1.6374,1.9614
2020-07-19,20200719OBHT0,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2

In [ ]:
df['초말']

2016-05-11    B
2016-05-11    T
2016-05-11    B
2016-05-11    T
2016-05-11    T
             ..
2020-07-19    B
2020-07-19    B
2020-07-19    T
2020-07-19    B
2020-07-19    T
Name: 초말, Length: 4562, dtype: object

In [ ]:
df = df.drop(labels=['게임키'],axis=1)
df

,팀코드,상대팀코드,더블헤더코드_상대최근_타자,초말,타자_상대최근_타자,타수_상대최근_타자,타점_상대최근,득점_상대최근,안타_상대최근_타자,2루타_상대최근_타자,3루타_상대최근_타자,홈런_상대최근_타자,도루_상대최근_타자,도루실패_상대최근_타자,희타_상대최근_타자,희비_상대최근_타자,4구_상대최근_타자,고4_상대최근_타자,사구_상대최근_타자,삼진_상대최근_타자,병살타_상대최근_타자,실책_상대최근_타자,잔루_상대최근,득점권타율_상대최근,득점권타수_상대최근,득점권안타_상대최근,시즌,단타_상대최근_타자,상대최근OBP,상대최근SLG,상대최근ops,상대최근타율,상대최근ISO,상대최근BABIP,상대최근BB/K,상대최근K%_타자,상대최근BB%_타자,상대최근wOBA,상대최근wRAA,상대최근wRC,...,최근kwERA,최근LOB,최근HR%_투수,최근K%_투수,최근BB%_투수,최근K9,최근BB9,최근HR9,최근KBB,최근whip,최근PFR,최근P/G,최근P/IP,최근P/PA,최근CYP,최근F-E,최근R-E,최근OAVG,최근OOBP,최근OSLG,최근OOPS,단타_최근_투수,최근투수AB,최근투수HIT,최근투수B1B,최근투수B2B,최근투수B3B,최근투수BB,최근투수HR,최근투수K,최근투수HBP,최근투수IBB,최근투수SB,최근투수CS,최근투수PA,최근투수INN,최근투수실점,최근득점권WHIP,최근2점차이하WHIP,최근345번타자WHIP
2016-05-11,HT,KT,0,B,40,38,6,8,14,6,0,1,3,0,1,0,1,0,0,6,0,1,8,0.375,16,6,2016,7,0.336207,0.372549,0.708756,0.254902,0.117647,0.312500,0.545455,0.189655,0.103448,0.328716,-3.519599,12.736589,...,4.538,0.676,0.031,0.154,0.082,6.328,3.164,1.266,2.000,1.594,1.078,148.2,17.367,3.800,2.333,0.469,1.055,0.297,0.369,0.463,0.832,7.0,35.0,10.4,7.6,1.0,0.6,3.0,1.2,6.0,0.8,0.2,1.4,0.6,39.0,8.533333,5.8,2.1428,4.6862,2.0002
2016-05-11,KT,HT,0,T,37,34,3,3,9,1,0,1,0,0,0,0,2,0,1,9,3,1,7,0.333,6,2,2016,7,0.353982,0.417476,0.771458,0.291262,0.126214,0.345679,0.400000,0.173913,0.069565,0.354469,-0.789482,15.326566,...,4.951,0.571,0.028,0.154,0.117,6.600,5.000,1.200,1.320,1.778,1.289,174.4,19.378,4.075,-3.200,-0.560,1.600,0.301,0.387,0.492,0.879,7.0,36.6,11.0,6.8,2.6,0.4,5.0,1.2,6.6,0.4,0.0,0.8,0.0,42.8,9.000000,8.0,2.7238,1.2890,2.9000
2016-05-11,HH,NC,0,B,40,36,5,6,13,4,0,0,0,0,1,0,3,0,0,7,1,3,10,0.333,12,4,2016,9,0.317308,0.269663,0.586971,0.202247,0.067416,0.288136,0.413793,0.266055,0.110092,0.290567,-7.097688,8.177523,...,5.668,0.542,0.045,0.116,0.138,5.850,6.975,2.250,0.839,2.750,1.425,167.0,20.875,3.728,-29.400,-3.652,1.350,0.418,0.498,0.619,1.117,8.0,33.4,11.4,9.4,0.8,0.2,5.8,1.0,5.6,0.2,0.0,0.8,0.6,40.4,8.400000,6.4,2.2822,1.9540,2.3050
2016-05-11,NC,HH,0,T,45,35,4,5,9,1,0,0,0,0,0,1,9,0,0,8,1,0,13,0.273,11,3,2016,8,0.351351,0.389474,0.740825,0.252632,0.136842,0.283784,0.684211,0.168142,0.115044,0.343180,-1.938596,13.897173,...,4.239,0.737,0.005,0.194,0.097,7.200,3.600,0.200,2.000,1.200,1.200,150.6,16.733,4.048,15.100,0.662,0.000,0.221,0.310,0.252,0.561,9.0,32.6,7.2,6.6,0.4,0.0,3.6,0.2,7.2,0.6,0.0,0.4,0.4,37.2,9.000000,3.2,1.5900,0.9470,1.1818
2016-05-11,OB,SK,0,T,41,35,7,7,12,2,0,3,1,1,0,1,5,0,0,6,2,0,7,0.143,7,1,2016,7,0.362832,0.539216,0.902048,0.303922,0.235294,0.294118,0.416667,0.104348,0.043478,0.394398,3.396350,19.512399,...,4.343,0.594,0.027,0.203,0.115,9.132,4.963,1.191,1.840,2.007,1.588,184.8,20.382,4.070,-14.733,-3.087,0.199,0.344,0.434,0.545,0.979,7.0,35.2,9.6,5.4,3.2,0.2,4.8,0.8,9.0,0.4,0.0,0.4,0.4,41.6,9.066667,5.6,1.7438,1.9974,1.8108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-19,SS,LT,0,B,34,31,1,1,7,0,0,1,2,1,0,0,3,0,0,9,1,0,6,0.000,5,0,2020,6,0.376068,0.500000,0.876068,0.317308,0.182692,0.318182,0.769231,0.109244,0.084034,0.388906,4.531493,13.586046,...,4.650,0.674,0.016,0.146,0.083,5.600,3.200,0.600,1.750,1.378,0.978,154.8,17.200,4.031,9.300,0.486,0.800,0.272,0.346,0.367,0.713,4.0,33.8,9.2,7.2,1.4,0.0,3.2,0.6,5.6,0.6,0.0,0.4,0.0,38.4,9.000000,4.8,1.7036,1.3554,1.5952
2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571,7,4,2020,7,0.370370,0.410526,0.780897,0.284211,0.126316,0.320988,0.923077,0.119266,0.100917,0.357150,1.159105,9.528860,...,5.147,0.754,0.026,0.132,0.111,5.153,4.328,1.031,1.190,1.580,1.053,150.4,17.221,3.958,3.333,0.849,0.000,0.304,0.387,0.424,0.811,7.0,31.6,9.6,7.8,0.8,0.0,4.2,1.0,5.0,0.6,0.0,0.6,0.6,38.0,8.733333,4.6,1.1722,1.6374,1.9614
2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.364,11,4,2020,7,0.377193,0.33

In [ ]:
cols = df.columns.tolist()
cols.index('결과')

128

In [ ]:
cols = df.columns.tolist()
tmp = cols[128] #11번째 column을
del cols[128]
cols.insert(0,tmp) #2번째 위치로
df = df[cols]

In [ ]:
from sklearn import preprocessing
def encoding(df):
    encoder = preprocessing.LabelEncoder()
    cols = ['팀코드','상대팀코드','초말','결과']
    for col in enumerate(cols):
        values = df[[col[1]]].values
        values = encoder.fit_transform(values) #범주형 팀코드, 상대팉코드, 초말, 결과 변수를 숫자로 인코딩 해 준다
        values = values.astype('float32')
        df[col[1]]=values
    return df

In [ ]:
df = encoding(df)
df

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,결과,팀코드,상대팀코드,더블헤더코드_상대최근_타자,초말,타자_상대최근_타자,타수_상대최근_타자,타점_상대최근,득점_상대최근,안타_상대최근_타자,2루타_상대최근_타자,3루타_상대최근_타자,홈런_상대최근_타자,도루_상대최근_타자,도루실패_상대최근_타자,희타_상대최근_타자,희비_상대최근_타자,4구_상대최근_타자,고4_상대최근_타자,사구_상대최근_타자,삼진_상대최근_타자,병살타_상대최근_타자,실책_상대최근_타자,잔루_상대최근,득점권타율_상대최근,득점권타수_상대최근,득점권안타_상대최근,시즌,단타_상대최근_타자,상대최근OBP,상대최근SLG,상대최근ops,상대최근타율,상대최근ISO,상대최근BABIP,상대최근BB/K,상대최근K%_타자,상대최근BB%_타자,상대최근wOBA,상대최근wRAA,...,최근kwERA,최근LOB,최근HR%_투수,최근K%_투수,최근BB%_투수,최근K9,최근BB9,최근HR9,최근KBB,최근whip,최근PFR,최근P/G,최근P/IP,최근P/PA,최근CYP,최근F-E,최근R-E,최근OAVG,최근OOBP,최근OSLG,최근OOPS,단타_최근_투수,최근투수AB,최근투수HIT,최근투수B1B,최근투수B2B,최근투수B3B,최근투수BB,최근투수HR,최근투수K,최근투수HBP,최근투수IBB,최근투수SB,최근투수CS,최근투수PA,최근투수INN,최근투수실점,최근득점권WHIP,최근2점차이하WHIP,최근345번타자WHIP
2016-05-11,1.0,1.0,2.0,0,0.0,40,38,6,8,14,6,0,1,3,0,1,0,1,0,0,6,0,1,8,0.375,16,6,2016,7,0.336207,0.372549,0.708756,0.254902,0.117647,0.312500,0.545455,0.189655,0.103448,0.328716,-3.519599,...,4.538,0.676,0.031,0.154,0.082,6.328,3.164,1.266,2.000,1.594,1.078,148.2,17.367,3.800,2.333,0.469,1.055,0.297,0.369,0.463,0.832,7.0,35.0,10.4,7.6,1.0,0.6,3.0,1.2,6.0,0.8,0.2,1.4,0.6,39.0,8.533333,5.8,2.1428,4.6862,2.0002
2016-05-11,0.0,2.0,1.0,0,1.0,37,34,3,3,9,1,0,1,0,0,0,0,2,0,1,9,3,1,7,0.333,6,2,2016,7,0.353982,0.417476,0.771458,0.291262,0.126214,0.345679,0.400000,0.173913,0.069565,0.354469,-0.789482,...,4.951,0.571,0.028,0.154,0.117,6.600,5.000,1.200,1.320,1.778,1.289,174.4,19.378,4.075,-3.200,-0.560,1.600,0.301,0.387,0.492,0.879,7.0,36.6,11.0,6.8,2.6,0.4,5.0,1.2,6.6,0.4,0.0,0.8,0.0,42.8,9.000000,8.0,2.7238,1.2890,2.9000
2016-05-11,1.0,0.0,5.0,0,0.0,40,36,5,6,13,4,0,0,0,0,1,0,3,0,0,7,1,3,10,0.333,12,4,2016,9,0.317308,0.269663,0.586971,0.202247,0.067416,0.288136,0.413793,0.266055,0.110092,0.290567,-7.097688,...,5.668,0.542,0.045,0.116,0.138,5.850,6.975,2.250,0.839,2.750,1.425,167.0,20.875,3.728,-29.400,-3.652,1.350,0.418,0.498,0.619,1.117,8.0,33.4,11.4,9.4,0.8,0.2,5.8,1.0,5.6,0.2,0.0,0.8,0.6,40.4,8.400000,6.4,2.2822,1.9540,2.3050
2016-05-11,0.0,5.0,0.0,0,1.0,45,35,4,5,9,1,0,0,0,0,0,1,9,0,0,8,1,0,13,0.273,11,3,2016,8,0.351351,0.389474,0.740825,0.252632,0.136842,0.283784,0.684211,0.168142,0.115044,0.343180,-1.938596,...,4.239,0.737,0.005,0.194,0.097,7.200,3.600,0.200,2.000,1.200,1.200,150.6,16.733,4.048,15.100,0.662,0.000,0.221,0.310,0.252,0.561,9.0,32.6,7.2,6.6,0.4,0.0,3.6,0.2,7.2,0.6,0.0,0.4,0.4,37.2,9.000000,3.2,1.5900,0.9470,1.1818
2016-05-11,1.0,6.0,7.0,0,1.0,41,35,7,7,12,2,0,3,1,1,0,1,5,0,0,6,2,0,7,0.143,7,1,2016,7,0.362832,0.539216,0.902048,0.303922,0.235294,0.294118,0.416667,0.104348,0.043478,0.394398,3.396350,...,4.343,0.594,0.027,0.203,0.115,9.132,4.963,1.191,1.840,2.007,1.588,184.8,20.382,4.070,-14.733,-3.087,0.199,0.344,0.434,0.545,0.979,7.0,35.2,9.6,5.4,3.2,0.2,4.8,0.8,9.0,0.4,0.0,0.4,0.4,41.6,9.066667,5.6,1.7438,1.9974,1.8108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-19,0.0,8.0,4.0,0,0.0,34,31,1,1,7,0,0,1,2,1,0,0,3,0,0,9,1,0,6,0.000,5,0,2020,6,0.376068,0.500000,0.876068,0.317308,0.182692,0.318182,0.769231,0.109244,0.084034,0.388906,4.531493,...,4.650,0.674,0.016,0.146,0.083,5.600,3.200,0.600,1.750,1.378,0.978,154.8,17.200,4.031,9.300,0.486,0.800,0.272,0.346,0.367,0.713,4.0,33.8,9.2,7.2,1.4,0.0,3.2,0.6,5.6,0.6,0.0,0.4,0.0,38.4,9.000000,4.8,1.7036,1.3554,1.5952
2020-07-19,0.0,1.0,6.0,0,0.0,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571,7,4,2020,7,0.370370,0.410526,0.780897,0.284211,0.126316,0.320988,0.923077,0.119266,0.100917,0.357150,1.159105,...,5.147,0.754,0.026,0.132,0.111,5.153,4.328,1.031,1.190,1.580,1.053,150.4,17.221,3.958,3.333,0.849,0.000,0.304,0.387,0.424,0.811,7.0,31.6,9.6,7.8,0.8,0.0,4.2,1.0,5.0,0.6,0.0,0.6,0.6,38.0,8.733333,4.6,1.1722,1.6374,1.9614
2020-07-19,1.0,6.0,1.0,0,1.0,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.364,11,4,2020,7,0.377193,0.336735,0.71

In [ ]:
df['결과']

2016-05-11    1.0
2016-05-11    0.0
2016-05-11    1.0
2016-05-11    0.0
2016-05-11    1.0
             ... 
2020-07-19    0.0
2020-07-19    0.0
2020-07-19    1.0
2020-07-19    1.0
2020-07-19    0.0
Name: 결과, Length: 4562, dtype: float32

In [ ]:
def split_by_team(df):
    df_list = [0]*10
    for i in range(len(df_list)):
        df_list[i] = df[df['팀코드']==i]
        df_list[i]
    return df_list
df_list = split_by_team(df)
df_list


[             결과  팀코드  상대팀코드  ...  최근득점권WHIP  최근2점차이하WHIP  최근345번타자WHIP
 2016-05-11  1.0  0.0    5.0  ...     2.2822       1.9540        2.3050
 2016-05-12  0.0  0.0    5.0  ...     2.2522       2.0066        2.5050
 2016-05-13  0.0  0.0    1.0  ...     2.5950       2.2800        2.6350
 2016-05-14  0.0  0.0    1.0  ...     2.2950       2.5408        2.2600
 2016-05-17  0.0  0.0    8.0  ...     2.4900       1.9796        2.1750
 ...         ...  ...    ...  ...        ...          ...           ...
 2020-07-14  0.0  0.0    2.0  ...     1.3148       1.4670        2.0608
 2020-07-15  1.0  0.0    2.0  ...     1.2482       1.3170        1.6000
 2020-07-16  0.0  0.0    2.0  ...     1.3482       1.4490        1.5250
 2020-07-18  0.0  0.0    3.0  ...     1.5594       1.8814        1.6892
 2020-07-19  0.0  0.0    3.0  ...     1.2808       1.9384        1.7892
 
 [464 rows x 269 columns],
              결과  팀코드  상대팀코드  ...  최근득점권WHIP  최근2점차이하WHIP  최근345번타자WHIP
 2016-05-11  1.0  1.0    2.0  ...  

In [ ]:
train_data = np.array(None)
val_data = np.array(None)
test_data = np.array(None)
for i in range(len(df_list)):
    train_data= np.append(train_data,np.asarray(df_list[i][(df_list[i]['시즌']!= 2019) & (df_list[i]['시즌']!= 2020)].drop('시즌', axis=1)).astype(np.float32))
    val_data= np.append(np.asarray(df_list[i][df_list[i]['시즌']==2019].drop('시즌', axis=1)).astype(np.float32))
    test_data= np.asarray(df_list[i][df_list[i]['시즌']==2020].drop('시즌', axis=1)).astype(np.float32)


ValueError: ignored

In [ ]:
train_data= np.asarray(df[(df['시즌']!= 2019) & (df['시즌']!= 2020)].drop('시즌', axis=1)).astype(np.float32)
val_data= np.asarray(df[df['시즌']==2019].drop('시즌', axis=1)).astype(np.float32)
test_data= np.asarray(df[df['시즌']==2020].drop('시즌', axis=1)).astype(np.float32)

In [ ]:
def make_dataset(data, seq_length=5, target_delay=0, strides=1):
    sequence = []
    target = []
    for index in range(len(data) - seq_length - target_delay):
        if index % strides == 0:
          sequence.append(data[index:index+seq_length])
          target.append(data[index+seq_length+target_delay][0])
    return np.array(sequence), np.array(target)

In [ ]:
def scaling(df):
    scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
    cols = df.columns.drop(labels='시즌')
    for col in enumerate(cols):
        values = df[[col[1]]].values
        scaled = scaler.fit_transform(values) #값을 0~1로 떨어뜨린다
        df[col[1]]=scaled
    return df

In [ ]:
for i in range(len(df_list)):
    df_list[i] = scaling(df_list[i])
df_list[1]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,결과,팀코드,상대팀코드,더블헤더코드_상대최근_타자,초말,타자_상대최근_타자,타수_상대최근_타자,타점_상대최근,득점_상대최근,안타_상대최근_타자,2루타_상대최근_타자,3루타_상대최근_타자,홈런_상대최근_타자,도루_상대최근_타자,도루실패_상대최근_타자,희타_상대최근_타자,희비_상대최근_타자,4구_상대최근_타자,고4_상대최근_타자,사구_상대최근_타자,삼진_상대최근_타자,병살타_상대최근_타자,실책_상대최근_타자,잔루_상대최근,득점권타율_상대최근,득점권타수_상대최근,득점권안타_상대최근,시즌,단타_상대최근_타자,상대최근OBP,상대최근SLG,상대최근ops,상대최근타율,상대최근ISO,상대최근BABIP,상대최근BB/K,상대최근K%_타자,상대최근BB%_타자,상대최근wOBA,상대최근wRAA,...,최근kwERA,최근LOB,최근HR%_투수,최근K%_투수,최근BB%_투수,최근K9,최근BB9,최근HR9,최근KBB,최근whip,최근PFR,최근P/G,최근P/IP,최근P/PA,최근CYP,최근F-E,최근R-E,최근OAVG,최근OOBP,최근OSLG,최근OOPS,단타_최근_투수,최근투수AB,최근투수HIT,최근투수B1B,최근투수B2B,최근투수B3B,최근투수BB,최근투수HR,최근투수K,최근투수HBP,최근투수IBB,최근투수SB,최근투수CS,최근투수PA,최근투수INN,최근투수실점,최근득점권WHIP,최근2점차이하WHIP,최근345번타자WHIP
2016-05-11,1.0,0.0,0.222222,0.0,0.0,0.500000,0.586207,0.400000,0.533333,0.666667,0.750,0.0,0.2,0.6,0.000000,0.333333,0.0,0.076923,0.00,0.00,0.333333,0.00,0.2,0.4375,0.375,0.80,0.666667,2016,0.411765,0.657132,0.490888,0.557831,0.559785,0.387543,0.683140,0.363636,0.588417,0.517241,0.618842,0.292485,...,0.582053,0.487113,0.534483,0.311881,0.398551,0.362030,0.371757,0.523573,0.158047,0.582993,0.367,0.439024,0.544393,0.430909,0.503758,0.660897,0.458297,0.612022,0.619289,0.575198,0.633205,0.35,0.545455,0.595745,0.605263,0.235294,0.75,0.37500,0.500000,0.342857,0.571429,0.333333,0.538462,0.6,0.418605,0.358974,0.536585,0.567115,0.449052,0.471971
2016-05-12,1.0,0.0,0.222222,0.0,0.0,0.576923,0.586207,0.666667,0.666667,0.777778,0.625,0.5,0.4,0.0,0.333333,0.000000,0.0,0.230769,0.25,0.25,0.600000,0.50,0.0,0.4375,0.333,0.75,0.555556,2016,0.470588,0.746859,0.682353,0.708328,0.725490,0.617647,0.850129,0.285714,0.525434,0.362903,0.753073,0.509570,...,0.479144,0.404639,0.534483,0.440594,0.362319,0.511168,0.338575,0.523573,0.218186,0.524939,0.461,0.421951,0.522129,0.416364,0.472957,0.556036,0.366638,0.551913,0.553299,0.559367,0.596525,0.25,0.545455,0.553191,0.526316,0.294118,0.75,0.34375,0.500000,0.485714,0.571429,0.333333,0.461538,0.6,0.406977,0.358974,0.560976,0.621129,0.436663,0.400138
2016-05-13,1.0,0.0,0.000000,0.0,0.0,0.230769,0.206897,0.200000,0.266667,0.222222,0.125,0.0,0.0,0.0,0.666667,0.000000,0.0,0.307692,0.25,0.50,0.066667,0.00,0.0,0.2500,0.111,0.45,0.111111,2016,0.294118,0.579125,0.334051,0.432737,0.433030,0.185584,0.617796,0.222222,0.919278,0.432099,0.500679,0.190840,...,0.555159,0.360825,0.689655,0.391089,0.456522,0.481340,0.404938,0.698098,0.173082,0.678659,0.508,0.453659,0.563535,0.371818,0.368234,0.498630,0.366638,0.688525,0.670051,0.709763,0.750965,0.25,0.581818,0.659574,0.605263,0.294118,0.75,0.40625,0.666667,0.457143,0.285714,0.666667,0.461538,0.4,0.476744,0.358974,0.682927,0.639134,0.453085,0.478255
2016-05-14,1.0,0.0,0.000000,0.0,0.0,0.500000,0.517241,0.466667,0.533333,0.555556,0.125,0.0,0.2,0.2,0.000000,0.000000,0.0,0.230769,0.00,0.25,0.133333,0.00,0.2,0.4375,0.385,0.65,0.555556,2016,0.588235,0.617225,0.336134,0.449324,0.448179,0.168067,0.584998,0.307692,0.713864,0.442478,0.522054,0.143651,...,0.517563,0.579897,0.465517,0.425743,0.427536,0.429036,0.327882,0.426385,0.200986,0.346688,0.411,0.382927,0.418952,0.485455,0.663922,0.757681,0.357950,0.344262,0.375635,0.408971,0.418919,0.25,0.363636,0.361702,0.342105,0.294118,0.50,0.34375,0.416667,0.428571,0.285714,0.666667,0.230769,0.6,0.313953,0.435897,0.365854,0.353930,0.070010,0.305430
2016-05-17,0.0,0.0,0.666667,0.0,1.0,0.461538,0.517241,0.200000,0.200000,0.500000,0.500,0.0,0.0,0.4,0.000000,0.000000,0.5,0.076923,0.00,0.25,0.600000,0.25,0.0,0.5000,0.200,0.50,0.222222,2016,0.411765,0.639984,0.465835,0.535961,0.554565,0.332739,0.628488,0.385965,0.521670,0.486726,0.596686,0.261347,...,0.503293,0.706186,0.275862,0.420792,0.384058,0.400481,0.314515,0.248139,0.200986,0.388389,0.356,0.412195,0.388949,0.506364,0.718686,0.675898,0.000000,0.475410,0.436548,0.290237,0.355212,0.35,0.436364,0.468085,0.605263,0.235294,0.00,0.34375,0.250000,0.428571,0.142857,0.333333,0.230769,0.4,0.337209,0.538462,0.268293,0.159100,0.063883,0.167798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
print("shape of train data: {}".format(train_data.shape))
print("shape of val data: {}".format(val_data.shape))
print("shape of test data: {}".format(test_data.shape))

shape of train data: (3026, 268)
shape of val data: (978, 268)
shape of test data: (558, 268)


In [ ]:
train_sequences, train_labels = make_dataset(train_data)
val_sequences, val_labels = make_dataset(val_data)
test_sequences, test_labels = make_dataset(test_data)

In [ ]:
print("shape of train sequences: {}".format(train_sequences.shape))
print("shape of val sequences: {}".format(val_sequences.shape))
print("shape of test sequences: {}".format(test_sequences.shape))

shape of train sequences: (3021, 5, 268)
shape of val sequences: (973, 5, 268)
shape of test sequences: (553, 5, 268)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

print('tensorflow version: {}'.format(tf.__version__))
print('GPU 사용 가능 여부: {}'.format(tf.test.is_gpu_available()))

batch_size = 9
max_epochs = 30
learning_rate = 4e-5
hidden_sizes = [128, 128]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_sequences, train_labels))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.repeat()

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_sequences, val_labels))
val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.repeat()

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_sequences, test_labels))
test_dataset = test_dataset.batch(batch_size)

In [ ]:
model = tf.keras.Sequential()

In [ ]:
num_layers = len(hidden_sizes)
for i in range(num_layers - 1):
    model.add(tf.keras.layers.LSTM(128,return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001),dropout=0.2))
    model.add(tf.keras.layers.LSTM(64,return_sequences=False,kernel_regularizer=tf.keras.regularizers.l2(0.001),dropout=0.2))
    model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01) ,loss='binary_crossentropy',metrics=['binary_accuracy'])

In [ ]:
loss, acc = model.evaluate(test_sequences, test_labels)
print("학습 전 모델의 loss: {:.5f}".format(loss))
print("학습 전 모델의 정확도: {:5.2f}%".format(100*acc))

18/18 [==============================] - 0s 4ms/step - loss: 1.2199 - binary_accuracy: 0.4991
학습 전 모델의 loss: 1.21991
학습 전 모델의 정확도: 49.91%


In [ ]:
import os
checkpoint_path = './checkpoint_z_score/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                             save_best_only=True,
                             save_weights_only=True,
                             verbose=1
                            )

In [ ]:
model.fit(train_dataset, steps_per_epoch = train_data.shape[1]/batch_size, 
          validation_data = val_dataset, 
          validation_steps= val_data.shape[1]/batch_size,
          epochs= max_epochs,
          callbacks= [checkpoint])

Epoch 1/30
24/29 [=======================>......] - ETA: 0s - loss: 1.2202 - binary_accuracy: 0.5046
Epoch 00001: val_loss improved from inf to 1.21634, saving model to ./checkpoint_z_score/cp.ckpt
30/29 [==============================] - 0s 11ms/step - loss: 1.2208 - binary_accuracy: 0.4963 - val_loss: 1.2163 - val_binary_accuracy: 0.4852
Epoch 2/30
25/29 [========================>.....] - ETA: 0s - loss: 1.2192 - binary_accuracy: 0.4711
Epoch 00002: val_loss improved from 1.21634 to 1.21542, saving model to ./checkpoint_z_score/cp.ckpt
30/29 [==============================] - 0s 11ms/step - loss: 1.2191 - binary_accuracy: 0.4630 - val_loss: 1.2154 - val_binary_accuracy: 0.5185
Epoch 3/30
23/29 [======================>.......] - ETA: 0s - loss: 1.2177 - binary_accuracy: 0.4686
Epoch 00003: val_loss improved from 1.21542 to 1.21476, saving model to ./checkpoint_z_score/cp.ckpt
30/29 [==============================] - 0s 11ms/step - loss: 1.2174 - binary_accuracy: 0.4667 - val_loss: 1.2

In [ ]:
for sequences, targets in train_dataset.take(1):
    outputs = model(sequences)
    print(outputs)

tf.Tensor(
[[0.4916072 ]
 [0.5018209 ]
 [0.4968017 ]
 [0.4996024 ]
 [0.5047144 ]
 [0.50226796]
 [0.5020581 ]
 [0.5009128 ]
 [0.5002743 ]], shape=(9, 1), dtype=float32)


In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
_, test_loss = model.evaluate(test_dataset, steps=len(test_sequences)/batch_size)

62/61 [==============================] - 0s 4ms/step - loss: 1.1954 - binary_accuracy: 0.5841


In [ ]:
loss, acc = model.evaluate(test_sequences, test_labels)
print("학습 후 모델의 loss: {:.5f}".format(loss))
print("학습 후 모델의 정확도: {:5.2f}%".format(100*acc))

18/18 [==============================] - 0s 4ms/step - loss: 1.1954 - binary_accuracy: 0.5841
학습 후 모델의 loss: 1.19537
학습 후 모델의 정확도: 58.41%


In [ ]:
train_data.shape

(3026, 268)